(COMPC)=

# 5.3 Compilación a C

```{admonition} Notas para contenedor de docker:

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker y `<versión imagen de docker>` por la versión más actualizada que se presenta en la documentación.

`docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_optimizacion_2 -p 8888:8888 -d palmoreck/jupyterlab_optimizacion_2:<versión imagen de docker>`

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

`docker stop jupyterlab_optimizacion_2`

Documentación de la imagen de docker `palmoreck/jupyterlab_optimizacion_2:<versión imagen de docker>` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/optimizacion_2).

```

---

```{admonition} Al final de esta nota la comunidad lectora:
:class: tip

* Comprenderá diferencias entre lenguajes de programación que son intérpretes y los que requieren/realizan pasos de compilación.

* Comprenderá por qué definir tipo de valores en lenguajes que son intérpretes conducen a tiempos de ejecución menores.

* Aprenderá lo que es una compilación *ahead of time* (AOT) y *just in time* (JIT). Se mostrarán ejemplos de lenguajes y paquetes que realizan ambos tipos de compilaciones.

```

Se presentan códigos y sus ejecuciones en una máquina `m4.16xlarge` con una AMI `ubuntu 20.04 - ami-042e8287309f5df03` de la nube de [AWS](https://aws.amazon.com/). Se utilizó en la sección de `User data` el [script_profiling_and_BLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_profiling_and_BLAS.sh)

La máquina `m4.16xlarge` tiene las siguientes características:

In [1]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          64
On-line CPU(s) list:             0-63
Thread(s) per core:              2
Core(s) per socket:              16
Socket(s):                       2
NUMA node(s):                    2
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Stepping:                        1
CPU MHz:                         3000.000
CPU max MHz:                     3000.0000
CPU min MHz:                     1200.0000
BogoMIPS:                        4600.00
Hypervisor vendor:               Xen
Virtualization type:             full
L1d cache:                       1 MiB
L1i cache:                       1 MiB
L2 cache:                        8

In [2]:
%%bash
sudo lshw -C memory

  *-firmware
       description: BIOS
       vendor: Xen
       physical id: 0
       version: 4.11.amazon
       date: 08/24/2006
       size: 96KiB
       capabilities: pci edd
  *-memory
       description: System Memory
       physical id: 1000
       size: 256GiB
       capabilities: ecc
       configuration: errordetection=multi-bit-ecc
     *-bank:0
          description: DIMM RAM
          physical id: 0
          slot: DIMM 0
          size: 16GiB
          width: 64 bits
     *-bank:1
          description: DIMM RAM
          physical id: 1
          slot: DIMM 1
          size: 16GiB
          width: 64 bits
     *-bank:2
          description: DIMM RAM
          physical id: 2
          slot: DIMM 2
          size: 16GiB
          width: 64 bits
     *-bank:3
          description: DIMM RAM
          physical id: 3
          slot: DIMM 3
          size: 16GiB
          width: 64 bits
     *-bank:4
          description: DIMM RAM
          physical id: 4
          slot: DIMM

In [3]:
%%bash
uname -ar #r for kernel, a for all

Linux ip-10-0-2-123 5.11.0-1022-aws #23~20.04.1-Ubuntu SMP Mon Nov 15 14:03:19 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux


```{admonition} Observación
:class: tip

En la celda anterior se utilizó el comando de *magic* `%%bash`. Algunos comandos de *magic* los podemos utilizar también con `import`. Ver [ipython-magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html#)

```

## Características de los lenguajes de programación

Los lenguajes de programación y sus implementaciones tienen características como las siguientes:

* Realizar un *parsing* de las instrucciones y ejecutarlas de forma casi inmediata (intérprete). Como ejemplo está el lenguaje: [Beginners' All-purpose Symbolic Instruction Code: BASIC](https://en.wikipedia.org/wiki/BASIC)

* Realizar un *parsing* de las instrucciones, traducirlas a una [representación intermedia](https://en.wikipedia.org/wiki/Intermediate_representation) (IR) y ejecutarlas. La traducción a una representación intermedia es un [bytecode](https://en.wikipedia.org/wiki/Bytecode). Como ejemplo se encuentra el lenguaje *Python* en su implementación [CPython](https://github.com/python/cpython).

* Compilar [ahead of time](https://en.wikipedia.org/wiki/Ahead-of-time_compilation) (AOT) las instrucciones antes de su ejecución. Como ejemplo se encuentran los lenguajes *C, C++* y *Fortran*.

* Realizar un *parsing* de las instrucciones y compilarlas en una forma [just in time compilation](https://en.wikipedia.org/wiki/Just-in-time_compilation) (JIT) *at* [runtime](https://en.wikipedia.org/wiki/Runtime_(program_lifecycle_phase)). Como ejemplos se encuentran los lenguajes *Julia* y *Python* en su implementación con [PyPy](https://doc.pypy.org/en/latest/index.html).

La ejecución de instrucciones será más rápida dependiendo del lenguaje, la implementación que se haga del mismo y de sus *features*.

```{admonition} Comentarios

* Varios proyectos están en desarrollo para mejorar eficiencia y otros temas. Algunos de ellos son:

    * *PyPy*

    * *A better API for extending Python in C*: [hpyproject](https://github.com/hpyproject/hpy)

* La implementación *CPython* de *Python* es la estándar, pero hay otras más como *PyPy*. Ver [python-vs-cpython](https://stackoverflow.com/questions/17130975/python-vs-cpython) para una breve explicación de implementaciones de Python. Ver [Alternative R implementations](http://adv-r.had.co.nz/Performance.html#faster-r) y [R implementations](https://en.wikipedia.org/wiki/R_(programming_language)#Implementations) para implementaciones de *R* diferentes a la estándar.

```

## Cpython

<img src="https://dl.dropboxusercontent.com/s/6quwf6c2ci5ey0n/cpython.png?dl=0" heigth="900" width="900">

## Compilación AOT y JIT

```{margin}

Es común utilizar la palabra librería en lugar de paquete en el contexto de compilación.

```

Una compilación AOT crea una librería, especializada para nuestras máquinas y se puede utilizar de forma instantánea. Un ejemplo de lo anterior lo tenemos con *Cython*, el cual es un paquete que realiza la compilación de módulos de *Python*. Por ejemplo, las librerías de *NumPy*, *SciPy* o *Scikit-learn* instalados vía *pip* o *conda* utilizan *Cython* para compilar secciones de tales librerías adaptadas a nuestras máquinas.

Una compilación JIT no requiere que se realice "trabajo previo" de nuestro lado, la compilación se realiza al tiempo que se utiliza el código, *at runtime*. En términos coloquiales, en una compilación JIT, se iniciará la ejecución del código identificando diferentes secciones que pueden compilarse y que por tanto se ejecutarán más lentamente de lo normal pues se estará realizando la compilación al tiempo de ejecución. Sin embargo, en sucesivas ejecuciones del **mismo** código tales secciones serán más rápidas. En resúmen se requiere un *warm-up*, ver por ejemplo [how-fast-is-pypy](https://doc.pypy.org/en/latest/faq.html#how-fast-is-pypy).

La compilación AOT da los mejores *speedups* pero solicita mayor trabajo de nuestro lado. La compilación JIT da buenos *speedups* con poca intervención nuestra pero utiliza más memoria y más tiempo en iniciar la ejecución del código, ver por ejemplo [python_performance-slide-15](https://raw.githubusercontent.com/vstinner/talks/main/2019-EuroPython/python_performance.pdf) acerca de *PyPy issues*. 

Para la ejecución frecuente de *scripts* pequeños la compilación AOT resulta una mejor opción que la compilación JIT, ver por ejemplo [couldn't the jit dump and reload already compiled machine code](https://doc.pypy.org/en/latest/faq.html#couldn-t-the-jit-dump-and-reload-already-compiled-machine-code). 

A continuación se presentan ejecuciones en diferentes lenguajes con sus implementaciones estándar para aproximar el área debajo de la curva de $f(x) = e^{-x^2}$ en el intervalo $[0, 1]$ con la regla del rectángulo compuesto. Se mide el tiempo de ejecución utilizando $n = 10^7$ nodos.

## Python

In [4]:
%%file Rcf_python.py
import math
import time
def Rcf(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (float): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat = (b-a)/n
    sum_res = 0
    for i in range(n):
        x = a+(i+1/2)*h_hat
        sum_res += f(x)
    return h_hat*sum_res

if __name__ == "__main__":   
    n = 10**7
    f = lambda x: math.exp(-x**2)
    a = 0
    b = 1
    start_time = time.time()
    res = Rcf(f,a,b,n)
    end_time = time.time()
    secs = end_time-start_time
    print("Rcf tomó", secs, "segundos" )

Writing Rcf_python.py


In [5]:
%%bash
python3 Rcf_python.py

Rcf tomó 3.4967801570892334 segundos


## R

In [6]:
%%file Rcf_R.R
Rcf<-function(f,a,b,n){
    '
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (float): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    '
    
    h_hat <- (b-a)/n
    sum_res <- 0
    for(i in 0:(n-1)){
        x <- a+(i+1/2)*h_hat
        sum_res <- sum_res + f(x)
    }
    approx <- h_hat*sum_res
}
n <- 10**7
f <- function(x)exp(-x^2)
a <- 0
b <- 1
system.time(Rcf(f,a,b,n))

Writing Rcf_R.R


In [7]:
%%bash
Rscript Rcf_R.R

   user  system elapsed 
  5.607   0.018   5.626 


## Julia

Ver: [Julia: performance-tips](https://docs.julialang.org/en/v1/manual/performance-tips/)

In [8]:
%%file Rcf_julia.jl
"""
Compute numerical approximation using rectangle or mid-point
method in an interval.

# Arguments

- `f::Float`: function expression of integrand.
- `a::Float`: left point of interval.
- `b::Float`: right point of interval.
- `n::Integer`: number of subintervals.
"""
function Rcf(f, a, b, n)
    h_hat = (b-a)/n
    sum_res = 0
    for i in 0.0:n-1
        x = a+(i+1/2)*h_hat
        sum_res += f(x)
    end    
    return h_hat*sum_res
end
function main()
    a = 0
    b = 1
    n =10^7
    f(x) = exp(-x^2)
    res(f, a, b, n) = @time Rcf(f, a, b, n)
    println(res(f, a, b, n))
    println(res(f, a, b, n))
end

main()

Writing Rcf_julia.jl


In [9]:
%%bash
/usr/local/julia-1.7.1/bin/julia Rcf_julia.jl

  0.239888 seconds (2.63 k allocations: 142.609 KiB, 41.51% compilation time)
0.7468241328123898
  0.140050 seconds (2 allocations: 32 bytes)
0.7468241328123898


(RCFJULIATYPEDVALUES)=

`Rcf_julia_typed_values.jl`

In [10]:
%%file Rcf_julia_typed_values.jl
"""
Compute numerical approximation using rectangle or mid-point
method in an interval.

# Arguments

- `f::Float`: function expression of integrand.
- `a::Float`: left point of interval.
- `b::Float`: right point of interval.
- `n::Integer`: number of subintervals.
"""
function Rcf(f, a, b, n)
    h_hat = (b-a)/n
    sum_res = 0.0
    for i in 0:n-1
        x = a+(i + 1/2)*h_hat
        sum_res += f(x)
    end    
    return h_hat*sum_res
end
function main()
    a = 0.0
    b = 1.0
    n =10^7
    f(x) = exp(-x^2)
    res(f, a, b, n) = @time Rcf(f, a, b, n)
    println(res(f, a, b, n))
    println(res(f, a, b, n))
end

main()

Writing Rcf_julia_typed_values.jl


In [11]:
%%bash
/usr/local/julia-1.7.1/bin/julia Rcf_julia_typed_values.jl

  0.143176 seconds (503 allocations: 26.594 KiB, 11.97% compilation time)
0.7468241328123898
  0.125599 seconds (4 allocations: 64 bytes)
0.7468241328123898


(RCFJULIANAIVE)=

`Rcf_julia_naive.jl`

In [12]:
%%file Rcf_julia_naive.jl
"""
Compute numerical approximation using rectangle or mid-point
method in an interval.

# Arguments

- `f::Float`: function expression of integrand.
- `a::Float`: left point of interval.
- `b::Float`: right point of interval.
- `n::Integer`: number of subintervals.
"""
function Rcf(f, a, b, n)
    h_hat = (b-a)/n
    sum_res = 0
    for i in 0:n-1
        x = a+(i + 1/2)*h_hat
        sum_res += f(x)
    end    
    return h_hat*sum_res
end
function main()
    a = 0
    b = 1
    n =10^7
    f(x) = exp(-x^2)
    res(f, a, b, n) = @time Rcf(f, a, b, n)
    println(res(f, a, b, n))
    println(res(f, a, b, n))
end

main()

Writing Rcf_julia_naive.jl


In [13]:
%%bash
/usr/local/julia-1.7.1/bin/julia Rcf_julia_naive.jl

  0.113146 seconds (588 allocations: 31.391 KiB, 17.07% compilation time)
0.7468241328123898
  0.094084 seconds (2 allocations: 32 bytes)
0.7468241328123898


## C

Para la medición de tiempos se utilizaron las ligas: [measuring-time-in-millisecond-precision](https://stackoverflow.com/questions/16764276/measuring-time-in-millisecond-precision) y [find-execution-time-c-program](https://www.techiedelight.com/find-execution-time-c-program/).

(RCFC)=

`Rcf_c.c`

In [9]:
%%file Rcf_c.c
#include<stdio.h>
#include<stdlib.h>
#include<math.h>
#include<time.h>
#include <sys/time.h>

void Rcf(double ext_izq, double ext_der, int n,\
    double *sum_res_p);
double f(double nodo);

int main(int argc, char *argv[]){
    double sum_res = 0.0;
    double a = 0.0, b = 1.0;
    int n = 1e7;
    struct timeval start;
    struct timeval end;
    long seconds;
    long long mili;
    
    gettimeofday(&start, NULL);
    Rcf(a,b,n,&sum_res);
    gettimeofday(&end, NULL);
    seconds = (end.tv_sec - start.tv_sec);
    mili = 1000*(seconds) + (end.tv_usec - start.tv_usec)/1000;    
    printf("Tiempo de ejecución: %lld milisegundos", mili);
    
    return 0;
}
void Rcf(double a, double b, int n, double *sum){
    double h_hat = (b-a)/n;
    double x = 0.0;
    int i = 0;
    *sum = 0.0;
    for(i = 0; i <= n-1; i++){
        x = a+(i+1/2.0)*h_hat;
        *sum += f(x);
    }
    *sum = h_hat*(*sum);
}
double f(double nodo){
    double valor_f;
    valor_f = exp(-pow(nodo,2));
    return valor_f;
}


Writing Rcf_c.c


In [10]:
%%bash
gcc -Wall Rcf_c.c -o Rcf_c.out -lm

In [11]:
%%bash
./Rcf_c.out

Tiempo de ejecución: 478 milisegundos

## ¿Por qué dar información sobre el tipo de valores (u objetos) que se utilizan en un código ayuda a que su ejecución sea más rápida?

*Python* es *dynamically typed* que se refiere a que un objeto de cualquier tipo y cualquier *statement* que haga referencia a un objeto, **pueden cambiar su tipo**. Esto hace difícil que la máquina virtual pueda optimizar la ejecución del código pues no se conoce qué tipo será utilizado para las operaciones futuras. Por ejemplo:

In [12]:
v = -1.0

In [13]:
print(type(v), abs(v))

<class 'float'> 1.0


In [14]:
v = 1 - 1j

In [15]:
print(type(v), abs(v))

<class 'complex'> 1.4142135623730951


La función `abs` trabaja diferente dependiendo del tipo de objeto. Para un número entero o punto flotante regresa el negativo de $-1.0$  y para un número complejo calcula una norma Euclidiana tomando de $v$ su parte real e imaginaria: $\text{abs}(v) = \sqrt{v.real^2 + v.imag^2}$.

Lo anterior en la práctica implica la ejecución de más instrucciones y por tanto mayor tiempo en ejecutarse. Antes de llamar a `abs` en la variable, *Python* revisa el tipo y decide cuál método llamar (*overhead*).

```{admonition} Comentarios

* Además cada número en *Python* está *wrapped up* en un objeto de *Python* de alto nivel. Por ejemplo para un entero se tiene el objeto `int`. Tal objeto tiene otras funciones por ejemplo `__str__` para imprimirlo.

* Es muy común que en los códigos no cambien los tipos por lo que la compilación AOT es una buena opción para una ejecución más rápida.

* Siguiendo con los dos comentarios anteriores, si sólo se desea calcular operaciones matemáticas (como el caso de la raíz cuadrada anterior) no requerimos la funcionalidad del objeto de alto nivel.

```

## [Cython](https://github.com/cython/cython/)

* Es un compilador que traduce instrucciones **anotadas** y escritas en un lenguaje híbrido entre Python y C que resultan un módulo compilado. Este módulo puede ser importado como un módulo regular de Python utilizando `import`. Típicamente el módulo compilado resulta ser similar en sintaxis al lenguaje *C*.

```{margin}

La frase código tipo *CPU-bound* es código cuya ejecución involucra un porcentaje mayor para uso de CPU que uso de memoria o I/O.

```

* Tiene un buen tiempo en la comunidad (2007 aproximadamente), es altamente usado y es de las herramientas preferidas para código tipo *CPU-bound*. Es un *fork* de [Pyrex](https://www.csse.canterbury.ac.nz/greg.ewing/python/Pyrex/) (2002) que expande sus capacidades.

```{admonition} Comentario

*Pyrex* en términos simples es *Python* con manejo de tipo de valores de *C*. *Pyrex* traduce el código escrito en *Python* a código de *C* (lo cual evita el uso de la [Python/C API](https://docs.python.org/3/c-api/index.html)) y permite la declaración de parámetros o valores en tipos de valores de *C*.

```

* Requiere conocimiento del lenguaje *C* lo cual debe tomarse en cuenta en un equipo de desarrollo de *software* y se sugiere utilizarlo en secciones pequeñas del código.

* Soporta la [API OpenMP](https://www.openmp.org/) para aprovechar los múltiples *cores* de una máquina.

* Puede utilizarse vía un script `setup.py` que compila un módulo para usarse con `import` y también puede utilizarse en *IPython* vía un comando *magic*.

<img src="https://dl.dropboxusercontent.com/s/162u0zcfpm8lewu/cython.png?dl=0" heigth="900" width="900">

```{admonition} Comentario

En el paso de compilación a código de máquina del dibujo anterior se omitieron detalles como son: creación de un archivo `.c`  y compilación de tal archivo con el compilador [gcc](https://gcc.gnu.org/) al módulo compilado (en sistemas Unix tiene extensión `.so`).

Ver [machine code](https://en.wikipedia.org/wiki/Machine_code)

```

* *Cython* y el compilador *gcc* analizan el código anotado para determinar qué instrucciones pueden optimizarse mediante una compilación AOT.

### ¿En qué casos y qué tipo de ganancias en velocidad podemos esperar al usar Cython?

* Un caso es en el que se tenga un código con muchos *loops* que realicen operaciones matemáticas típicamente no vectorizadas o que no pueden vectorizarse. Esto es, códigos en los que las instrucciones son básicamente sólo *Python* sin utilizar paquetes externos. Además, si en el ciclo las variables no cambian de su tipo (por ejemplo de `int` a `float`) entonces es un código que obtendrá ganancia en velocidad al compilar a código de máquina.

```{admonition} Observación
:class: tip

Si tu código de *Python* llama a operaciones vectorizadas vía *NumPy* podría ser que no se ejecute más rápido tu código después de compilarlo. Principalmente porque probablemente no se crearán muchos objetos intermedios que es un *feature* de *NumPy*.

```

* No esperamos tener un *speedup* después de compilar para llamadas a librerías externas (por ejemplo paqueterías que manejan bases de datos). También es poco probable que se obtengan ganancias significativas en programas que tengan alta carga de I/O.

* En general es poco probable que tu código compilado se ejecute más rápido que un código en *C* "bien escrito" y también es poco probable que se ejecute más lento. Es muy posible que el código *C* generado desde *Python* mediante *Cython* pueda alcanzar las velocidades de un código escrito en *C*, a menos que la persona que programó en *C* tenga un gran conocimiento de formas de hacer que el código de *C* se ajuste a la arquitectura de la máquina sobre la que se ejecutan los códigos.

### Ejemplo utilizando un archivo `setup.py`

In [1]:
import math
import time

from pytest import approx
from scipy.integrate import quad
from IPython.display import HTML, display

Para este caso requerimos tres archivos:

1.El código que será compilado en un archivo con extensión `.pyx` (escrito en *Python*). 

```{admonition} Observación
:class: tip

La extensión `.pyx` se utiliza en el lenguaje *Pyrex*. 
```

2.Un archivo `setup.py` que contiene las instrucciones para llamar a *Cython* y se encarga de crear el módulo compilado.

3.El código escrito en *Python* que importará el módulo compilado.

Archivo `.pyx`:

In [2]:
%%file Rcf_cython.pyx
def Rcf(f,a,b,n): #Rcf: rectángulo compuesto para f
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (float): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat = (b-a)/n
    nodes = [a+(i+1/2)*h_hat for i in range(n)]
    sum_res = 0
    for node in nodes:
        sum_res = sum_res+f(node)
    return h_hat*sum_res

Writing Rcf_cython.pyx


Archivo `setup.py` que contiene las instrucciones para el *build*:

In [3]:
%%file setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize("Rcf_cython.pyx", 
                              compiler_directives={'language_level' : 3})
     )

Writing setup.py


Compilar desde la línea de comandos:

In [4]:
%%bash
python3 setup.py build_ext --inplace

Compiling Rcf_cython.pyx because it changed.
[1/1] Cythonizing Rcf_cython.pyx
running build_ext
building 'Rcf_cython' extension
creating build
creating build/temp.linux-x86_64-3.8
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c Rcf_cython.c -o build/temp.linux-x86_64-3.8/Rcf_cython.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/Rcf_cython.o -o /home/ubuntu/analisis-numerico-computo-cientifico/libro_optimizacion/temas/V.optimizacion_de_codigo/5.3/Rcf_cython.cpython-38-x86_64-linux-gnu.so


Importar módulo compilado y ejecutarlo:

In [5]:
f=lambda x: math.exp(-x**2) #using math library

In [6]:
n = 10**7
a = 0
b = 1

In [7]:
import Rcf_cython

In [8]:
start_time = time.time()
res = Rcf_cython.Rcf(f, a, b,n)
end_time = time.time()

In [9]:
secs = end_time-start_time
print("Rcf tomó",secs,"segundos" )

Rcf tomó 3.731260061264038 segundos


In [10]:
obj, err = quad(f, a, b)

In [11]:
print(res == approx(obj))

True


### Comando de *magic* `%cython`

```{margin}

Ver [extensions-bundled-with-ipython](https://ipython.readthedocs.io/en/stable/config/extensions/index.html?highlight=cython#extensions-bundled-with-ipython) para extensiones que antes se incluían en *Ipython*.

```

Al instalar *Cython* se incluye tal comando. Al ejecutarse crea el archivo `.pyx`, lo compila con `setup.py` e importa en el *notebook*. 

In [12]:
%load_ext Cython

In [13]:
%%cython
def Rcf(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (float): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat = (b-a)/n
    nodes = [a+(i+1/2)*h_hat for i in range(n)]
    sum_res = 0
    for node in nodes:
        sum_res = sum_res+f(node)
    return h_hat*sum_res

In [14]:
start_time = time.time()
res = Rcf(f, a, b,n)
end_time = time.time()

In [15]:
secs = end_time-start_time
print("Rcf tomó",secs,"segundos" )

Rcf tomó 3.7382779121398926 segundos


In [16]:
obj, err = quad(f, a, b)

In [17]:
print(res == approx(obj))

True


### Anotaciones para analizar un bloque de código

*Cython* tiene la opción de *annotation* para generar un archivo con extensión `.html` en el que cada línea puede ser expandida haciendo un doble click que mostrará el código *C* generado. Líneas "más amarillas" refieren a más llamadas en la máquina virtual de *Python*, mientras que líneas más blancas significan "más código en *C* y no *Python*".

El objetivo es remover la mayor cantidad de líneas amarillas posibles pues son costosas en tiempo. Si tales líneas están dentro de loops serán todavía más costosas. Al final se busca tener códigos cuyas anotaciones sean lo más blancas posibles. 

```{admonition} Observación
:class: tip

Concentra tu atención en las líneas que son amarillas y están dentro de los *loops*, no inviertas tiempo en líneas amarillas que están fuera de *loops* y que no causan una ejecución lenta. Una ayuda para identificar lo anterior la da el perfilamiento.

```

### Ejemplo vía línea de comando

In [18]:
%%bash
$HOME/.local/bin/cython --force -3 --annotate Rcf_cython.pyx

Ver archivo creado: `Rcf_cython.html`

```{margin}

La liga correcta del archivo `Rcf_cython.c` es [Rcf_cython.c](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/libro_optimizacion/temas/V.optimizacion_de_codigo/5.3/Rcf_cython.c)

```

In [19]:
display(HTML("Rcf_cython.html"))

```{admonition} Comentarios

Para el código anterior el *statement* en donde se crean los nodos involucra un *loop* y es "muy amarilla". Si se perfila el código se verá que es una línea en la que se gasta una buena parte del tiempo total de ejecución del código.

```

Una primera opción que tenemos es crear los nodos para el método de integración dentro del *loop* y separar el llamado a la *list comprehension* `nodes=[a+(i+1/2)*h_hat for i in range(n)]`:

In [20]:
%%file Rcf_2_cython.pyx
def Rcf(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (float): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat = (b-a)/n
    sum_res = 0
    for i in range(n):
        x = a+(i+1/2)*h_hat
        sum_res += f(x)
    return h_hat*sum_res

Writing Rcf_2_cython.pyx


In [21]:
%%bash
$HOME/.local/bin/cython --force -3 --annotate Rcf_2_cython.pyx

```{margin}

La liga correcta del archivo `Rcf_2_cython.c` es [Rcf_2_cython.c](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/libro_optimizacion/temas/V.optimizacion_de_codigo/5.3/Rcf_2_cython.c)

```

In [22]:
display(HTML("Rcf_2_cython.html"))

```{admonition} Comentario

Para el código anterior los *statements* que están dentro del loop son "muy amarillos". En tales *statements* involucran tipos de valores que no cambiarán en la ejecución de cada *loop*. Una opción es **declarar los tipos de objetos** que están involucrados en el loop utilizando la sintaxis `cdef`. Ver [function_declarations](https://notes-on-cython.readthedocs.io/en/latest/function_declarations.html), [definition-of-def-cdef-and-cpdef-in-cython](https://stackoverflow.com/questions/28362009/definition-of-def-cdef-and-cpdef-in-cython/41976772)

```

In [23]:
%%file Rcf_3_cython.pyx
def Rcf(f, double a, double b, unsigned int n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (float): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    cdef unsigned int i
    cdef double x, sum_res, h_hat
    h_hat = (b-a)/n
    sum_res = 0
    for i in range(n):
        x = a+(i+1/2)*h_hat
        sum_res += f(x)
    return h_hat*sum_res

Writing Rcf_3_cython.pyx


In [24]:
%%bash
$HOME/.local/bin/cython -3 --force --annotate Rcf_3_cython.pyx

```{margin}

La liga correcta del archivo `Rcf_3_cython.c` es [Rcf_3_cython.c](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/libro_optimizacion/temas/V.optimizacion_de_codigo/5.3/Rcf_3_cython.c)

```

In [25]:
display(HTML("Rcf_3_cython.html"))

```{admonition} Comentario

Al definir tipos, éstos sólo serán entendidos por *Cython* y no por *Python*. Cython utiliza estos tipos para convertir el código de *Python* a código de *C*.

```

Una opción con la que perdemos flexibilidad pero ganamos en disminuir tiempo de ejecución es directamente llamar a la función `math.exp`:

In [26]:
%%file Rcf_4_cython.pyx
import math
def Rcf(double a, double b, unsigned int n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    cdef unsigned int i
    cdef double x, sum_res, h_hat
    h_hat = (b-a)/n
    sum_res = 0
    for i in range(n):
        x = a+(i+1/2)*h_hat
        sum_res += math.exp(-x**2)
    return h_hat*sum_res

Writing Rcf_4_cython.pyx


In [27]:
%%bash
$HOME/.local/bin/cython -3 --force --annotate Rcf_4_cython.pyx

```{margin}

La liga correcta del archivo `Rcf_4_cython.c` es [Rcf_4_cython.c](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/libro_optimizacion/temas/V.optimizacion_de_codigo/5.3/Rcf_4_cython.c)

```

In [28]:
display(HTML("Rcf_4_cython.html"))

Mejoramos el tiempo si directamente utilizamos la función `exp` de la librería `math` de *Cython*, ver [calling C functions](https://cython.readthedocs.io/en/latest/src/tutorial/external.html).

(RCF5CYTHON)=

`Rcf_5_cython.pyx`

In [29]:
%%file Rcf_5_cython.pyx
from libc.math cimport exp as c_exp

cdef double f(double x) nogil:
    return c_exp(-x**2)
    
def Rcf(double a, double b, unsigned int n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
            
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    cdef unsigned int i
    cdef double x, sum_res, h_hat
    h_hat = (b-a)/n
    sum_res = 0
    for i in range(n):
        x = a+(i+1/2)*h_hat
        sum_res += f(x)
    return h_hat*sum_res

Writing Rcf_5_cython.pyx


In [30]:
%%bash
$HOME/.local/bin/cython -3 --force --annotate Rcf_5_cython.pyx

```{margin}

La liga correcta del archivo `Rcf_5_cython.c` es [Rcf_5_cython.c](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/libro_optimizacion/temas/V.optimizacion_de_codigo/5.3/Rcf_5_cython.c)

```

In [31]:
display(HTML("Rcf_5_cython.html"))

```{admonition} Comentario

Un *tradeoff* en la optimización de código se realiza entre flexibilidad, legibilidad y una ejecución rápida del código.

```

In [32]:
%%file setup_2.py
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize("Rcf_2_cython.pyx", 
                              compiler_directives={'language_level' : 3})
     )

Writing setup_2.py


Compilar desde la línea de comandos:

In [33]:
%%bash
python3 setup_2.py build_ext --inplace

running build_ext
building 'Rcf_2_cython' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c Rcf_2_cython.c -o build/temp.linux-x86_64-3.8/Rcf_2_cython.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/Rcf_2_cython.o -o /home/ubuntu/analisis-numerico-computo-cientifico/libro_optimizacion/temas/V.optimizacion_de_codigo/5.3/Rcf_2_cython.cpython-38-x86_64-linux-gnu.so


In [34]:
%%file setup_3.py
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize("Rcf_3_cython.pyx", 
                              compiler_directives={'language_level' : 3})
     )

Writing setup_3.py


Compilar desde la línea de comandos:

In [35]:
%%bash
python3 setup_3.py build_ext --inplace

running build_ext
building 'Rcf_3_cython' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c Rcf_3_cython.c -o build/temp.linux-x86_64-3.8/Rcf_3_cython.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/Rcf_3_cython.o -o /home/ubuntu/analisis-numerico-computo-cientifico/libro_optimizacion/temas/V.optimizacion_de_codigo/5.3/Rcf_3_cython.cpython-38-x86_64-linux-gnu.so


In [36]:
%%file setup_4.py
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize("Rcf_4_cython.pyx", 
                              compiler_directives={'language_level' : 3})
     )

Writing setup_4.py


Compilar desde la línea de comandos:

In [37]:
%%bash
python3 setup_4.py build_ext --inplace

running build_ext
building 'Rcf_4_cython' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c Rcf_4_cython.c -o build/temp.linux-x86_64-3.8/Rcf_4_cython.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/Rcf_4_cython.o -o /home/ubuntu/analisis-numerico-computo-cientifico/libro_optimizacion/temas/V.optimizacion_de_codigo/5.3/Rcf_4_cython.cpython-38-x86_64-linux-gnu.so


In [38]:
%%file setup_5.py
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize("Rcf_5_cython.pyx", 
                              compiler_directives={'language_level' : 3})
     )

Writing setup_5.py


Compilar desde la línea de comandos:

In [39]:
%%bash
python3 setup_5.py build_ext --inplace

running build_ext
building 'Rcf_5_cython' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c Rcf_5_cython.c -o build/temp.linux-x86_64-3.8/Rcf_5_cython.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/Rcf_5_cython.o -o /home/ubuntu/analisis-numerico-computo-cientifico/libro_optimizacion/temas/V.optimizacion_de_codigo/5.3/Rcf_5_cython.cpython-38-x86_64-linux-gnu.so


Importar módulos compilados:

In [40]:
import Rcf_2_cython, Rcf_3_cython, Rcf_4_cython, Rcf_5_cython

In [41]:
start_time = time.time()
res_2 = Rcf_2_cython.Rcf(f, a, b,n)
end_time = time.time()

In [42]:
secs = end_time-start_time
print("Rcf_2 tomó",secs,"segundos" )

Rcf_2 tomó 3.3402740955352783 segundos


Verificamos que después de la optimización de código continuamos resolviendo correctamente el problema:

In [43]:
print(res_2 == approx(obj))

True


In [44]:
start_time = time.time()
res_3 = Rcf_3_cython.Rcf(f, a, b,n)
end_time = time.time()

In [45]:
secs = end_time-start_time
print("Rcf_3 tomó",secs,"segundos" )

Rcf_3 tomó 2.496501922607422 segundos


In [46]:
print(res_3 == approx(obj))

True


In [47]:
start_time = time.time()
res_4 = Rcf_4_cython.Rcf(a, b,n)
end_time = time.time()

In [48]:
secs = end_time-start_time
print("Rcf_4 tomó",secs,"segundos" )

Rcf_4 tomó 0.7289412021636963 segundos


In [49]:
print(res_4 == approx(obj))

True


In [50]:
start_time = time.time()
res_5 = Rcf_5_cython.Rcf(a, b,n)
end_time = time.time()

In [51]:
secs = end_time-start_time
print("Rcf_5 tomó",secs,"segundos" )

Rcf_5 tomó 0.10629606246948242 segundos


Verificamos que después de la optimización de código continuamos resolviendo correctamente el problema:

In [52]:
print(res_5 == approx(obj))

True


### Ejemplo de implementación con *NumPy*

Comparamos con una implementación usando *NumPy* y vectorización:

In [53]:
import numpy as np

In [54]:
f_np = lambda x: np.exp(-x**2)

(RCFNUMPY)=

`Rcf_numpy`

In [55]:
def Rcf_numpy(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (float): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat = (b-a)/n
    aux_vec = np.linspace(a, b, n+1)
    nodes = (aux_vec[:-1]+aux_vec[1:])/2
    return h_hat*np.sum(f(nodes))

In [56]:
start_time = time.time()
res_numpy = Rcf_numpy(f_np, a, b,n)
end_time = time.time()

In [57]:
secs = end_time-start_time
print("Rcf_numpy tomó",secs,"segundos" )

Rcf_numpy tomó 0.273637056350708 segundos


In [58]:
print(res_numpy == approx(obj))

True


```{admonition} Comentarios

* La implementación con *NumPy* resulta ser la segunda más rápida principalmente por el uso de bloques contiguos de memoria para almacenar los valores y la vectorización. La implementación anterior, sin embargo, requiere un conocimiento de las funciones de tal paquete. Para este ejemplo utilizamos `linspace` y la funcionalidad de realizar operaciones de forma vectorizada para la creación de los nodos y evaluación de la función. Una situación que podría darse es que para un problema no podamos utilizar alguna función de *NumPy* o bien no tengamos el ingenio para pensar cómo realizar una operación de forma vectorizada. En este caso *Cython* puede ser una opción a utilizar.

* En *Cython* se tienen las [memoryviews](https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html) para acceso de bajo nivel a la memoria similar a la que proveen los *arrays* de *NumPy* en el caso de requerirse *arrays* en una forma más general que no sólo sean de *NumPy* (por ejemplo de *C* o de *Cython*, ver [Cython arrays](https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html#view-cython-arrays)).

```

```{admonition} Observación
:class: tip

Compárese la implementación vía *NumPy* con el uso de listas para los nodos. Recuérdese que las listas de *Python* alojan locaciones donde se pueden encontrar los valores y no los valores en sí. Los *arrays* de *NumPy* almacenan tipos de valores primitivos. Las listas tienen *data fragmentation* que causan *memory fragmentation* y por tanto un mayor impacto del *Von Neumann bottleneck*. Además el almacenamiento de tipo de objetos de alto nivel en las listas causa *overhead* en lugar de almacenamiento de tipo de valores primitivos en un *array* de *NumPy*.

```

### *Cython* y [OpenMP](http://www.openmp.org/)

*OpenMP* es una extensión al lenguaje *C* y es una API para cómputo en paralelo en un sistema de memoria compartida, *aka, shared memory parallel programming* con CPUs. Se revisará con mayor profundidad en la nota de cómputo en paralelo. 

```{margin}

Ver [global interpreter lock](https://docs.python.org/3.9/glossary.html#term-global-interpreter-lock) (GIL), [global interpreter lock](https://en.wikipedia.org/wiki/Global_interpreter_lock)

```

En *Cython*, *OpenMP* se utiliza mediante [prange](https://cython.readthedocs.io/en/latest/src/userguide/parallelism.html#cython.parallel.prange) (*parallel range*). Además debe deshabilitarse el *GIL*.

```{admonition} Observación
:class: tip

Al deshabilitar el GIL en una sección de código se debe operar con tipos primitivos. En tal sección no se debe operar con objetos *Python* (por ejemplo listas).

```

(RCF5CYTHONOPENMP)=

`Rcf_5_cython_openmp`

In [62]:
%%file Rcf_5_cython_openmp.pyx
from cython.parallel import prange
from libc.math cimport exp as c_exp

cdef double f(double x) nogil:
    return c_exp(-x**2)

def Rcf(double a, double b, unsigned int n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
            
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    cdef int i
    cdef double x, sum_res, h_hat
    h_hat = (b-a)/n
    sum_res = 0
    for i in prange(n, schedule="guided", nogil=True):
        x = a+(i+1/2)*h_hat
        sum_res += f(x)
    return h_hat*sum_res

Writing Rcf_5_cython_openmp.pyx


```{admonition} Comentario

Con `prange` puede elegirse diferente *scheduling*. Si `schedule` recibe el valor `static` el trabajo a realizar se reparte equitativamente entre los *cores* y si algunos *threads* terminan antes permanecerán sin realizar trabajo, *aka idle*. Con `dynamic` y `guided` se reparte de manera dinámica *at runtime* que es útil si la cantidad de trabajo es variable y si *threads* terminan antes pueden recibir trabajo a realizar.

```

In [63]:
%%bash
$HOME/.local/bin/cython -3 --force Rcf_5_cython_openmp.pyx

En el archivo `setup.py` se coloca la **directiva** `-fopenmp`.

```{margin}

Ver [Rcf_5_cython_openmp.c](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/libro_optimizacion/temas/V.optimizacion_de_codigo/5.3/Rcf_5_cython_openmp.c) para la implementación en *C* de la función `Rcf_5_cython_openmp.Rcf`.

```

In [64]:
%%file setup_5_openmp.py
from setuptools import Extension, setup
from Cython.Build import cythonize

ext_modules = [Extension("Rcf_5_cython_openmp",
                         ["Rcf_5_cython_openmp.pyx"], 
                         extra_compile_args=["-fopenmp"],
                         extra_link_args=["-fopenmp"],
                        )
              ]

setup(ext_modules = cythonize(ext_modules))

Writing setup_5_openmp.py


Compilar desde la línea de comandos:

In [65]:
%%bash
python3 setup_5_openmp.py build_ext --inplace

running build_ext
building 'Rcf_5_cython_openmp' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c Rcf_5_cython_openmp.c -o build/temp.linux-x86_64-3.8/Rcf_5_cython_openmp.o -fopenmp
creating build/lib.linux-x86_64-3.8
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/Rcf_5_cython_openmp.o -o build/lib.linux-x86_64-3.8/Rcf_5_cython_openmp.cpython-38-x86_64-linux-gnu.so -fopenmp
copying build/lib.linux-x86_64-3.8/Rcf_5_cython_openmp.cpython-38-x86_64-linux-gnu.so -> 


In [66]:
import Rcf_5_cython_openmp

In [67]:
start_time = time.time()
res_5_openmp = Rcf_5_cython_openmp.Rcf(a, b, n)
end_time = time.time()

In [68]:
secs = end_time-start_time
print("Rcf_5_openmp tomó",secs,"segundos" )

Rcf_5_openmp tomó 0.017746686935424805 segundos


Verificamos que después de la optimización de código continuamos resolviendo correctamente el problema:

In [69]:
print(res_5_openmp == approx(obj))

True


```{admonition} Ejercicio
:class: tip

Implementar la regla de Simpson compuesta con *NumPy*, *Cython* y *Cython* + *OpenMP* en una máquina de AWS con las mismas características que la que se presenta en esta nota y medir tiempo de ejecución.

```

## [Numba](https://github.com/numba/numba)

* Utiliza compilación JIT *at runtime* mediante el compilador [llvmlite](https://github.com/numba/llvmlite).

* Puede utilizarse para funciones *built in* de *Python* o de *NumPy*.

* Tiene soporte para cómputo en paralelo en CPU/GPU.

* Utiliza [CFFI](https://cffi.readthedocs.io/en/latest/) y [ctypes](https://docs.python.org/3/library/ctypes.html) para llamar a funciones de *C*. 

* Ver [numba architecture](https://numba.readthedocs.io/en/stable/developer/architecture.html#architecture) para una explicación detallada de su funcionamiento.

Se utiliza un *decorator* para anotar cuál función se desea compilar.

### Ejemplo de uso con *Numba*

In [67]:
from numba import jit

```{margin}

En [glossary: nopython](https://numba.pydata.org/numba-doc/latest/glossary.html#term-nopython-mode) se da la definición de *nopython mode* en *Numba*. Ahí se indica que se genera código que no usa Python C API y requiere que los tipos de valores nativos de Python puedan ser inferidos. 

Puede usarse el *decorator* `njit` que es un *alias* para `@jit(nopython=True)`.

```

(RCFNUMBA)=

`Rcf_numba`

In [71]:
@jit(nopython=True)
def Rcf_numba(a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
            
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat = (b-a)/n
    sum_res = 0
    for i in range(n):
        x = a+(i+1/2)*h_hat
        sum_res += np.exp(-x**2)
    return h_hat*sum_res


In [72]:
start_time = time.time()
res_numba = Rcf_numba(a,b,n)
end_time = time.time()

```{margin}

Se mide dos veces el tiempo de ejecución para no incluir el tiempo de compilación. Ver [5minguide](https://numba.pydata.org/numba-doc/latest/user/5minguide.html).

```

In [73]:
secs = end_time-start_time
print("Rcf_numba con compilación tomó", secs, "segundos" )

Rcf_numba con compilación tomó 0.5527660846710205 segundos


In [74]:
start_time = time.time()
res_numba = Rcf_numba(a,b,n)
end_time = time.time()

In [75]:
secs = end_time-start_time
print("Rcf_numba tomó", secs, "segundos" )

Rcf_numba tomó 0.22369146347045898 segundos


Verificamos que después de la optimización de código continuamos resolviendo correctamente el problema:

In [76]:
print(res_numba == approx(obj))

True


Con la función [inspect_types](https://numba.readthedocs.io/en/stable/reference/jit-compilation.html?highlight=inspect_types#Dispatcher.inspect_types) nos ayuda para revisar si pudo inferirse información de los tipos de valores a partir del código escrito.

In [77]:
print(Rcf_numba.inspect_types())

Rcf_numba (int64, int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-71-541f7545b9ed>
# --- LINE 1 --- 

@jit(nopython=True)

# --- LINE 2 --- 

def Rcf_numba(a,b,n):

    # --- LINE 3 --- 

    """

    # --- LINE 4 --- 

    Compute numerical approximation using rectangle or mid-point

    # --- LINE 5 --- 

    method in an interval.

    # --- LINE 6 --- 

    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for

    # --- LINE 7 --- 

    i=0,1,...,n-1 and h_hat=(b-a)/n

    # --- LINE 8 --- 

    Args:

# --- LINE 9 --- 



        # --- LINE 10 --- 

        a (float): left point of interval.

# --- LINE 11 --- 



        # --- LINE 12 --- 

        b (float): right point of interval.

# --- LINE 13 --- 



        # --- LINE 14 --- 

        n (int): number of subintervals.

# --- LINE 15 --- 



    # --- LINE 16 --- 

    Returns:

# --- LINE 17 --- 



        # --- LINE 18 --- 

        sum_res (float)

### Ejemplo de uso de *Numba* con cómputo en paralelo 

Ver [numba: parallel](https://numba.pydata.org/numba-doc/latest/user/parallel.html#), [numba: threading layer](http://numba.pydata.org/numba-doc/latest/user/threading-layer.html)

In [78]:
from numba import prange

(RCFNUMBAPARALLEL)=

`Rcf_numba_parallel`

In [79]:
@jit(nopython=True, parallel=True)
def Rcf_numba_parallel(a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
            
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat = (b-a)/n
    sum_res = 0
    for i in prange(n):
        x = a+(i+1/2)*h_hat
        sum_res += np.exp(-x**2)
    return h_hat*sum_res


In [80]:
start_time = time.time()
res_numba_parallel = Rcf_numba_parallel(a,b,n)
end_time = time.time()

In [81]:
secs = end_time-start_time
print("Rcf_numba_parallel con compilación tomó", secs, "segundos" )

Rcf_numba_parallel con compilación tomó 1.074690580368042 segundos


In [82]:
start_time = time.time()
res_numba_parallel = Rcf_numba_parallel(a,b,n)
end_time = time.time()

```{margin} 

Ver [parallel-diagnostics](https://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics) para información relacionada con la ejecución en paralelo. Por ejemplo ejecutar `Rcf_numba_parallel.parallel_diagnostics(level=4)`.

```

In [84]:
secs = end_time-start_time
print("Rcf_numba_parallel tomó", secs, "segundos" )

Rcf_numba_parallel tomó 0.011192798614501953 segundos


Verificamos que después de la optimización de código continuamos resolviendo correctamente el problema:

In [86]:
print(res_numba_parallel == approx(obj))

True


### Ejemplo *Numpy* y *Numba*

En el siguiente ejemplo se utiliza la función `linspace` para auxiliar en la creación de los nodos y obsérvese que *Numba* sin problema trabaja los ciclos *for* (en el caso por ejemplo que no hubiéramos podido vectorizar la operación de creación de nodos).

In [5]:
@jit(nopython=True)
def Rcf_numpy_numba(a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
            
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat = (b-a)/n
    aux_vec = np.linspace(a, b, n+1)
    sum_res = 0
    for i in range(n-1):
        x = (aux_vec[i]+aux_vec[i+1])/2
        sum_res += np.exp(-x**2)
    return h_hat*sum_res

In [8]:
start_time = time.time()
res_numpy_numba = Rcf_numpy_numba(a, b,n)
end_time = time.time()

In [9]:
secs = end_time-start_time
print("Rcf_numpy_numba con compilación tomó",secs,"segundos" )

Rcf_numpy_numba con compilación tomó 0.5971698760986328 segundos


In [10]:
start_time = time.time()
res_numpy_numba = Rcf_numpy_numba(a, b,n)
end_time = time.time()

In [11]:
secs = end_time-start_time
print("Rcf_numpy_numba tomó",secs,"segundos" )

Rcf_numpy_numba tomó 0.1863393783569336 segundos


In [17]:
print(res_numpy_numba == approx(obj))

True


In [18]:
@jit(nopython=True)
def Rcf_numpy_numba_2(a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
            
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat = (b-a)/n
    aux_vec = np.linspace(a, b, n+1)
    nodes = (aux_vec[:-1]+aux_vec[1:])/2
    return h_hat*np.sum(np.exp(-nodes**2))

In [19]:
start_time = time.time()
res_numpy_numba_2 = Rcf_numpy_numba_2(a, b,n)
end_time = time.time()

In [20]:
secs = end_time-start_time
print("Rcf_numpy_numba_2 con compilación tomó",secs,"segundos" )

Rcf_numpy_numba_2 con compilación tomó 0.8459124565124512 segundos


In [21]:
start_time = time.time()
res_numpy_numba_2 = Rcf_numpy_numba_2(a, b,n)
end_time = time.time()

In [22]:
secs = end_time-start_time
print("Rcf_numpy_numba_2 tomó",secs,"segundos" )

Rcf_numpy_numba_2 tomó 0.26958131790161133 segundos


In [23]:
print(res_numpy_numba_2 == approx(obj))

True


```{admonition} Observación
:class: tip

Obsérvese que no se mejora el tiempo de ejecución utilizando vectorización y `linspace` que usando un ciclo for en la implementación anterior `Rcf_numpy_numba`. De hecho en `Rcf_numpy_numba_2` tiene un tiempo de ejecución igual que {ref}`Rcf_numpy <RCFNUMPY>`.


```

```{admonition} Ejercicio
:class: tip

Implementar la regla de Simpson compuesta con *Numba*, *Numpy* y *Numba*, *Numba* con cómputo en paralelo en una máquina de AWS con las mismas características que la que se presenta en esta nota y medir tiempo de ejecución.

```

## [Rcpp](https://github.com/RcppCore/Rcpp) 

*Rcpp* permite integrar *C++* y *R* de forma sencilla mediante su API.

### ¿Por qué usar Rcpp?

Con *Rcpp* nos da la posibilidad de obtener eficiencia en ejecución de un código con *C++* conservando la flexibilidad de trabajar con *R*. *C* o *C++* aunque requieren más líneas de código, son órdenes de magnitud más rápidos que *R*. Sacrificamos las ventajas que tiene *R* como facilidad de escribir códigos por velocidad en ejecución.

### ¿Cuando podríamos usar Rcpp?

* En *loops* que no pueden vectorizarse de forma sencilla. Si tenemos *loops* en los que una iteración depende de la anterior.

* Si hay que llamar una función millones de veces.

### ¿Por qué no usamos *C*?

Sí es posible llamar funciones de *C* desde *R* pero resulta en más trabajo por parte de nosotros. Por ejemplo, de acuerdo a H. Wickham:

*"...R’s C API. Unfortunately this API is not well documented. I’d recommend starting with my notes at [R’s C interface](http://adv-r.had.co.nz/C-interface.html). After that, read “[The R API](http://cran.rstudio.com/doc/manuals/r-devel/R-exts.html#The-R-API)” in “Writing R Extensions”. A number of exported functions are not documented, so you’ll also need to read the [R source code](https://github.com/wch/r-source) to figure out the details."*


Y como primer acercamiento a la compilación de código desde *R* es preferible seguir las recomendaciones de H. Wickham en utilizar la API de *Rcpp*.

### Ejemplo con *Rcpp*

En la siguiente implementación se utiliza [vapply](https://www.rdocumentation.org/packages/functools/versions/0.2.0/topics/Vapply) que es más rápida que [sapply](https://www.rdocumentation.org/packages/memisc/versions/0.99.27.3/topics/Sapply) pues se especifica con anterioridad el tipo de valor que devuelve.

In [1]:
Rcf <- function(f,a,b,n){
    '
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (float): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    '
    h_hat <- (b-a)/n
    sum_res <- 0
    x <- vapply(0:(n-1),function(j)a+(j+1/2)*h_hat,numeric(1))
    for(j in 1:n){
        sum_res <- sum_res+f(x[j])
    }
    h_hat*sum_res
}

In [2]:
a <- 0
b <- 1
f <- function(x)exp(-x^2)
n <- 10**7

In [3]:
system.time(res <- Rcf(f,a,b,n))

   user  system elapsed 
 13.988   0.111  14.100 

In [4]:
err_relativo <- function(aprox,obj)abs(aprox-obj)/abs(obj)

```{margin}

En la documentación de `integrate` se menciona que se utilice [Vectorize](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/Vectorize).
                                 
```

In [5]:
obj <- integrate(Vectorize(f),0,1) 

In [6]:
print(err_relativo(res,obj$value))

[1] 4.99495e-14


In [7]:
Rcf_2 <- function(f,a,b,n){
    '
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (float): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    '
    h_hat <- (b-a)/n
    x <- vapply(0:(n-1),function(j)a+(j+1/2)*h_hat,numeric(1))
    h_hat*sum(f(x))
}

In [8]:
system.time(res_2 <- Rcf_2(f,a,b,n))

   user  system elapsed 
 10.658   0.133  10.792 

In [9]:
print(err_relativo(res_2,obj$value))

[1] 2.973185e-16


In [10]:
library(Rcpp)

En *Rcpp* se tiene la función [cppFunction](https://www.rdocumentation.org/packages/Rcpp/versions/1.0.3/topics/cppFunction) que recibe código escrito en *C++* para definir una función que puede ser utilizada desde *R*. 

Primero reescribamos la implementación en la que no utilicemos `vapply`.

In [11]:
Rcf_3 <- function(f,a,b,n){
    '
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (float): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    '
    h_hat <- (b-a)/n
    sum_res <- 0
    for(i in 0:(n-1)){
        x <- a+(i+1/2)*h_hat
        sum_res <- sum_res+f(x)
    }
    h_hat*sum_res
}

In [12]:
system.time(res_3 <- Rcf_3(f,a,b,n))

   user  system elapsed 
  5.768   0.000   5.769 

In [13]:
print(err_relativo(res_3,obj$value))

[1] 4.99495e-14


(RCFRCPP)=

`Rcf_Rcpp`

Escribimos *source code* en *C++* que será el primer parámetro que recibirá `cppFunction`.

In [19]:
f_str <- 'double Rcf_Rcpp(double a, double b, int n){
             double h_hat;
             double sum_res=0;
             int i;
             double x;
             h_hat=(b-a)/n;
             for(i=0;i<=n-1;i++){
                    x = a+(i+1/2.0)*h_hat;
                    sum_res += exp(-pow(x,2));
             }
             return h_hat*sum_res;
         }'

In [20]:
cppFunction(f_str)

Si queremos obtener más información de la ejecución de la línea anterior podemos usar lo siguiente.

```{margin}

Se utiliza `rebuild=TRUE` para que se vuelva a compilar, ligar con la librería en *C++*  y más operaciones de `cppFunction`.

```

In [21]:
cppFunction(f_str, verbose=TRUE, rebuild=TRUE) 


Generated code for function definition: 
--------------------------------------------------------

#include <Rcpp.h>

using namespace Rcpp;

// [[Rcpp::export]]
double Rcf_Rcpp(double a, double b, int n){
             double h_hat;
             double sum_res=0;
             int i;
             double x;
             h_hat=(b-a)/n;
             for(i=0;i<=n-1;i++){
                    x = a+(i+1/2.0)*h_hat;
                    sum_res += exp(-pow(x,2));
             }
             return h_hat*sum_res;
         }

Generated extern "C" functions 
--------------------------------------------------------


#include <Rcpp.h>
// Rcf_Rcpp
double Rcf_Rcpp(double a, double b, int n);
RcppExport SEXP sourceCpp_4_Rcf_Rcpp(SEXP aSEXP, SEXP bSEXP, SEXP nSEXP) {
BEGIN_RCPP
    Rcpp::RObject rcpp_result_gen;
    Rcpp::RNGScope rcpp_rngScope_gen;
    Rcpp::traits::input_parameter< double >::type a(aSEXP);
    Rcpp::traits::input_parameter< double >::type b(bSEXP);
    Rcpp::traits::input_parameter< 

```{admonition} Comentarios

* Al ejecutar la línea de `cppFunction`, *Rcpp* compilará el código de *C++* y construirá una función de *R* que se conecta con la función compilada de *C++*. 

* Si se lee la salida de la ejecución con `verbose=TRUE` se utiliza un tipo de valor `SEXP`. De acuerdo a H. Wickham:

*...functions that talk to R must use the SEXP type for both inputs and outputs. SEXP, short for S expression, is the C struct used to represent every type of object in R. A C function typically starts by converting SEXPs to atomic C objects, and ends by converting C objects back to a SEXP. (The R API is designed so that these conversions often don’t require copying.)*

* La función `Rcpp::wrap` convierte objetos de *C++* a objetos de *R* y `Rcpp:as` viceversa.

```


In [22]:
system.time(res_4 <- Rcf_Rcpp(a,b,n))

   user  system elapsed 
  0.106   0.000   0.106 

In [23]:
print(err_relativo(res_4,obj$value))

[1] 4.99495e-14


### Otras funcionalidades de *Rcpp*

### `NumericVector`

En *Rcpp* se definen clases para relacionar tipos de valores de *R* con tipo de valores de *C++* para el manejo de vectores. Entre éstas se encuentran `NumericVector`, `IntegerVector`, `CharacterVector` y `LogicalVector` que se relacionan con vectores tipo `numeric`, `integer`, `character` y `logical` respectivamente. 

Por ejemplo, para el caso de `NumericVector` se tiene el siguiente ejemplo.

In [24]:
f_str <- 'NumericVector my_f(NumericVector x){
          return exp(log(x));
         }'

In [25]:
cppFunction(f_str)

In [26]:
print(my_f(seq(0,1,by=.1)))

 [1] 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0


### Ejemplo con `NumericVector`

Para mostrar otro ejemplo en el caso de la regla de integración del rectángulo considérese la siguiente implementación.

In [27]:
Rcf_implementation_example <- function(f,a,b,n){
    '
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (float): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (int): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    '
    h_hat <- (b-a)/n
    fx <- f(vapply(0:(n-1),function(j)a+(j+1/2)*h_hat,numeric(1)))
    h_hat*sum(fx)
}

In [28]:
res_numeric_vector <- Rcf_implementation_example(f,a,b,n)

In [29]:
print(err_relativo(res_numeric_vector,obj$value))

[1] 2.973185e-16


Utilicemos *Rcpp* para definir una función que recibe un `NumericVector` para realizar la suma.

```{margin}

El método `.size()` regresa un *integer*.

```

In [30]:
f_str<-'double Rcf_numeric_vector(NumericVector f_x,double h_hat){
             double sum_res=0;
             int i;
             int n = f_x.size();
             for(i=0;i<=n-1;i++){
                    sum_res+=f_x[i];
             }
             return h_hat*sum_res;
        }'

In [31]:
h_hat <- (b-a)/n

In [32]:
fx <- f(vapply(0:(n-1),function(j)a+(j+1/2)*h_hat,numeric(1)))

In [33]:
print(tail(fx))

[1] 0.3678798 0.3678798 0.3678797 0.3678796 0.3678796 0.3678795


In [34]:
cppFunction(f_str,rebuild=TRUE)

In [35]:
res_numeric_vector <- Rcf_numeric_vector(fx,h_hat)

In [36]:
print(err_relativo(res_numeric_vector,obj$value))

[1] 4.99495e-14


Otro ejemplo en el que se devuelve un vector tipo `NumericVector` para crear los nodos.

In [37]:
f_str <- 'NumericVector Rcf_nodes(double a, double b, int n){
          double h_hat=(b-a)/n;
          int i;
          NumericVector x(n);
          for(i=0;i<n;i++)
              x[i]=a+(i+1/2.0)*h_hat;
          return x;
         }'

In [38]:
cppFunction(f_str,rebuild=TRUE)

In [39]:
print(Rcf_nodes(0,1,2))

[1] 0.25 0.75


### Ejemplo de llamado a función definida en ambiente global con *Rcpp*

También en *Rcpp* es posible llamar funciones definidas en el ambiente global, por ejemplo.

```{margin}

`RObject` es una clase de *C++* para definir un objeto de *R*.

```

In [40]:
f_str <- 'RObject fun(double x){
          Environment env = Environment::global_env();
          Function f=env["f"];
          return f(x);
          }'

In [41]:
cppFunction(f_str,rebuild=TRUE)

In [42]:
fun(1)

[1] 0.3678794

In [43]:
f(1)

[1] 0.3678794

In [44]:
print(fun)

function (x) 
.Call(<pointer: 0x7fba201b25f0>, x)


```{admonition} Comentario

`.Call` es una función base para llamar funciones de `C` desde `R`:

*There are two ways to call C functions from R: .C() and .Call(). .C() is a quick and dirty way to call an C function that doesn’t know anything about R because .C() automatically converts between R vectors and the corresponding C types. .Call() is more flexible, but more work: your C function needs to use the R API to convert its inputs to standard C data types.*

**H. Wickham**.

```

In [45]:
print(f)

function(x)exp(-x^2)
<bytecode: 0x55669726fec8>


```{admonition} Ejercicio
:class: tip

Revisar [rcpp-sugar](http://adv-r.had.co.nz/Rcpp.html#rcpp-sugar), [Rcpp syntactic sugar](http://dirk.eddelbuettel.com/code/rcpp/Rcpp-sugar.pdf) y proponer programas que utilicen *sugar*.

```

```{admonition} Ejercicio
:class: tip

Implementar la regla de Simpson compuesta con *Rcpp* en una máquina de AWS con las mismas características que la que se presenta en esta nota y medir tiempo de ejecución.

```

```{admonition} Comentario

También existe el paquete [RcppParallel](https://github.com/RcppCore/RcppParallel) para cómputo en paralelo con la funcionalidad de *Rcpp*.

```

## Tabla resúmen con códigos que reportaron los mejores tiempos

En la siguiente tabla se presentan ligas hacia las implementaciones de la regla del rectángulo compuesta en diferentes lenguajes y paqueterías que reportaron los mejores tiempos. **Cualquier código siguiente presenta muy buen desempeño en tal implementación. Las diferencias de tiempos entre los códigos son pequeñas y se sugiere volver a correr los códigos midiendo tiempos con paqueterías que permitan la repetición de las mediciones como [timeit](https://docs.python.org/3/library/timeit.html)**.

|Lenguaje| Código |
|:---:|:---:|
|*Python*| {ref}`Rcf_numba_parallel <RCFNUMBAPARALLEL>`|
|*Python*| {ref}`Rcf_5_cython_openmp <RCF5CYTHONOPENMP>`|
|*Julia*| {ref}`Rcf_julia_naive <RCFJULIANAIVE>`|
|*R*| {ref}`Rcf_Rcpp <RCFRCPP>`|
|*Python*| {ref}`Rcf_5_cython <RCF5CYTHON>`|
|*Julia*| {ref}`Rcf_julia_typed_values <RCFJULIATYPEDVALUES>`|
|*Python*| {ref}`Rcf_numba <RCFNUMBA>`|
|*Python*| {ref}`Rcf_numpy <RCFNUMPY>`|
|*C*| {ref}`Rcf_c <RCFC>`|

```{admonition} Ejercicio
:class: tip

Presentar una tabla de resúmen de tiempos para las implementaciones pedidas en los ejercicios anteriores.

```

## Referencias de interés

* [Cython](https://en.wikipedia.org/wiki/Cython).

* [introduction to cython](http://okigiveup.net/an-introduction-to-cython/).

* [Basic cython tutorial](https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html)

* [Source files and compilation](https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html)

* [Compiling with a jupyter notebook](https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#compiling-with-a-jupyter-notebook)

* [hpy-kick-off-sprint-report](https://morepypy.blogspot.com/2019/12/hpy-kick-off-sprint-report.html)

* [PyPy - FAQ](https://doc.pypy.org/en/latest/faq.html)

* [Glossary: virtual machine](https://docs.python.org/3.9/glossary.html#term-virtual-machine)

* [Glossary: Cpython](https://docs.python.org/3.9/glossary.html#term-cpython)

* [Glossary: bytecode](https://docs.python.org/3.9/glossary.html#term-bytecode).

* [Python interpreter](https://docs.python.org/3/tutorial/interpreter.html), [what-is-interpreter-explain-how-python-interpreter-works](https://medium.com/@dpthegrey/what-is-interpreter-explain-how-python-interpreter-works-125205c1f8d6)

* [what-is-runtime-in-context-of-python-what-does-it-consist-of](https://stackoverflow.com/questions/60273813/what-is-runtime-in-context-of-python-what-does-it-consist-of)

* [numba: performance tips](https://numba.pydata.org/numba-doc/latest/user/performance-tips.html), [numba: generators](https://numba.pydata.org/numba-doc/latest/developer/generators.html)

* [Dirk Eddelbuettel: rcpp ](http://dirk.eddelbuettel.com/code/rcpp.html).

* [rcpp.org](http://www.rcpp.org/)

* [Rcpp for everyone](https://teuder.github.io/rcpp4everyone_en/).

* [Introduction to rcpp:From Simple Examples to Machine Learning](http://dirk.eddelbuettel.com/papers/rcpp_rfinance_may2017.pdf).

* [Rcpp note](http://yixuan.cos.name/rcpp-note/index.html).

* [Rcpp quick reference guide](http://dirk.eddelbuettel.com/code/rcpp/Rcpp-quickref.pdf).

* [should-i-prefer-rcppnumericvector-over-stdvector](https://stackoverflow.com/questions/41602024/should-i-prefer-rcppnumericvector-over-stdvector)

* [rcppparallel-getting-r-and-c-to-work-some-more-in-parallel](https://blog.rstudio.com/2016/01/15/rcppparallel-getting-r-and-c-to-work-some-more-in-parallel/)

* [Learncpp](https://www.learncpp.com/).

* [Cplusplus](http://www.cplusplus.com/).

* [what-are-the-downsides-of-jit-compilation](https://stackoverflow.com/questions/4037946/what-are-the-downsides-of-jit-compilation)


```{admonition} Ejercicios
:class: tip

1.Resuelve los ejercicios y preguntas de la nota.
```


**Referencias:**

1. M. Gorelick, I. Ozsvald, High Performance Python, O'Reilly Media, 2014.

2. [H. Wickham, Advanced R, 2014](http://adv-r.had.co.nz/Rcpp.html)

3. B. W. Kernighan, D. M. Ritchie, The C Programming Language, Prentice Hall Software Series, 1988

4. [C](https://github.com/palmoreck/programming-languages/tree/master/C)